<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-0:-All-imports-necessary" data-toc-modified-id="Part-0:-All-imports-necessary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 0: All imports necessary</a></span></li><li><span><a href="#Part-I:-Probabilistic-and-Statistical-Models-for-Outlier-Detection" data-toc-modified-id="Part-I:-Probabilistic-and-Statistical-Models-for-Outlier-Detection-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part I: Probabilistic and Statistical Models for Outlier Detection</a></span><ul class="toc-item"><li><span><a href="#Univariate-extreme-value-analysis" data-toc-modified-id="Univariate-extreme-value-analysis-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Univariate extreme-value analysis</a></span><ul class="toc-item"><li><span><a href="#Markov-inequality" data-toc-modified-id="Markov-inequality-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Markov inequality</a></span></li><li><span><a href="#Chebychev-inequality" data-toc-modified-id="Chebychev-inequality-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Chebychev inequality</a></span></li></ul></li></ul></li><li><span><a href="#Part-II:-Linear-models-for-Outlier-Detection" data-toc-modified-id="Part-II:-Linear-models-for-Outlier-Detection-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Part II: Linear models for Outlier Detection</a></span><ul class="toc-item"><li><span><a href="#One-Class-SVM" data-toc-modified-id="One-Class-SVM-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>One-Class SVM</a></span></li></ul></li><li><span><a href="#Part-III:-Proximity-Based-Outlier-Detection" data-toc-modified-id="Part-III:-Proximity-Based-Outlier-Detection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Part III: Proximity-Based Outlier Detection</a></span><ul class="toc-item"><li><span><a href="#The-Local-Outlier-Factor-(LOF)" data-toc-modified-id="The-Local-Outlier-Factor-(LOF)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>The Local Outlier Factor (LOF)</a></span></li></ul></li></ul></div>

# Part 0: All imports necessary

In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager

from sklearn import svm
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
np.random.seed(0)

# Part I: Probabilistic and Statistical Models for Outlier Detection

## Univariate extreme-value analysis

### Markov inequality

Tail inequalities can be used in order to bound the probability that a value in the tail of a probability distribution should be considered anomalous.

The strength of a tail inequality depends on the number of assumptions made about the underlying random variable.

Fewer assumptions lead to weaker inequalities but such inequalities apply to larger classes of random variables.

For example, the **Markov** and **Chebychev** inequalities are weak inequalities but they apply to very large classes of random variables.

On the other hand, the **Chernoff** bound and **Hoeffding** inequality are both stronger inequalities but they apply to restricted classes of random variables.

**Theorem**: Let X be a random variable that takes on only non-negative random values. Then, for any constant $\alpha$ satisfying $E[X] < \alpha$, the following
is true: $$P(X > \alpha) \leq \frac{E[X]}{\alpha}$$

So for example you have some sample from a random variable that takes on only non-negative random values (say Log-normally distributed):

In [ ]:
mean = 2
sigma = 0.7
sample = np.random.lognormal(mean=mean, sigma=sigma, size=10000)

count, bins, ignored = plt.hist(sample, 50, normed=1, facecolor='g', alpha=0.75)

x = np.linspace(min(bins), max(bins), 10000)
pdf = (np.exp(-(np.log(x) - mean)**2 / (2 * sigma**2)) / (x * sigma * np.sqrt(2 * np.pi)))

In [ ]:
plt.plot(x, pdf, linewidth=2, color='r')
plt.title('Some feature we need to clean from outliers')
plt.xlim((0, 140))
plt.show()

In [ ]:
values, base = np.histogram(sample, normed=True, bins=50)
cumulative = np.cumsum(values)

In [ ]:
plt.plot(base[:-1], len(sample)-cumulative)
plt.title('Some feature we need to clean from outliers')
plt.show()

To be able to apply Markov inequality you need to know the true mean value (E[X]).

In our case E[X] is equal to 2.

All we need to do is to decide what number will be the threshold ($\alpha$) for outlier detection.

Suppose $\alpha$ is equal to 20.

Then according to Markov inequality $$P(X > 20) \leq \frac{1}{10}$$

Thus if we agreed that $\frac{1}{10}$ is appropriate borderline between inliers and outliers then we can use it to detect the last ones.

Pros:
- only one assumption sould be verified (non-negativity);
- only one true value should be known (mean);

Cons:
- it gives a threshold only for upper-tail;
- it doesn't give a [supremum](https://en.wikipedia.org/wiki/Infimum_and_supremum) for upper-tail boundary.

### Chebychev inequality

The **Markov** inequality is defined only for probability distributions of non-negative values and provides a bound only on the upper tail.

In practice, it is often desired to bound both tails of arbitrary distributions.

Consider the case where X is an arbitrary random variable, which is not necessarily non-negative.

In such cases the **Markov** inequality cannot be used directly.

However, the (related) **Chebychev** inequality is very useful in such cases.

The **Chebychev** inequality is a direct application of the **Markov** inequality to a non-negative derivative of random variable X

**Theorem:** Let X be an arbitrary random variable. Then, for any constant $\alpha$, the following is true:
$$P(\lvert X - E[X] \rvert > \lvert \alpha \rvert) \leq \frac{Var[X]}{\alpha^2}$$

So again in our case:
- E[X] = 2;
- Var[X] = 0.7;
- $\alpha$ = 20.

Then according to **Chebychev** inequality $$P(\lvert X - 2 \rvert >  20) \leq \frac{0.7}{20^2} = \frac{7}{4000}$$

As you can see **Chebychev** inequality gives significantly smaller probability for the same $\alpha$.

Pros:
- random variable doesn't have to be non-negative;
- it gives a threshold for both upper-tail and lower-tail;

Cons:
- true values of E[X] and Var[X] should be known;
- it doesn't give a
[supremum](https://en.wikipedia.org/wiki/Infimum_and_supremum)
and
[infimum](https://en.wikipedia.org/wiki/Infimum_and_supremum)
for upper-tail and lower-tail boundaries.

# Part II: Linear models for Outlier Detection

## One-Class SVM

One-Class SVM is an example of Unsupervised Outlier Detection.

The main issue this method is trying to solve can be described in following words:

first look at our problem situation; we would like to determine whether (new) test data is member of a specific class, determined by our training data, or is not.

Why would we want this? Imagine a factory type of setting; heavy machinery under constant surveillance of some advanced system.

The task of the controlling system is to determine when something goes wrong; the products are below quality, the machine produces strange vibrations or something like a temperature that rises.

It is relatively easy to gather training data of situations that are OK; it is just the normal production situation.

But on the other side, collection example data of a faulty system state can be rather expensive, or just impossible.

If a faulty system state could be simulated, there is no way to guarantee that all the faulty states are simulated and thus recognized in a traditional two-class problem.

To cope with this problem, one-class classification problems (and solutions) are introduced.

By just providing the normal training data, an algorithm creates a (representational) model of this data.

If newly encountered data is too different, according to some measurement, from this model, it is labeled as out-of-class.

So first thing we need to do is to create a coordinate grid:

In [ ]:
xx, yy = np.meshgrid(np.linspace(-5, 5, 500), np.linspace(-5, 5, 500))

Then here we create a main dataset of "normal" (or usual) data:

In [ ]:
X = 0.3 * np.random.randn(100, 2)
X_train = np.r_[X + 2, X - 2]

We also need to generate a bunch on new (but still normal) data:

In [ ]:
X = 0.3 * np.random.randn(20, 2)
X_test = np.r_[X + 2, X - 2]

And here comes a bunch of outliers that was generated from another distribution:

In [ ]:
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))

Create an outlier detector:

In [ ]:
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(X_train)

Here we just generate our predictions for old usual data points, new usual data points and outliers:

In [ ]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)

And calculate the number of errors that were made during predictions generation:

In [ ]:
n_error_train = len(y_pred_train[y_pred_train == -1])
n_error_test = len(y_pred_test[y_pred_test == -1])
n_error_outliers = len(y_pred_outliers[y_pred_outliers == 1])

Here we return values of a decision function that we have got after fitting:

In [ ]:
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

The last part is to take a look at results:

In [ ]:
plt.title("Outlier/Novelty Detection with One-Class SVM")
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7))
a = plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='darkred')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')

s = 40
b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c='white', s=s, edgecolors='k')
b2 = plt.scatter(X_test[:, 0], X_test[:, 1], c='blueviolet', s=s, edgecolors='k')
c = plt.scatter(X_outliers[:, 0], X_outliers[:, 1], c='gold', s=s, edgecolors='k')
plt.axis('tight')
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.legend(
    [a.collections[0], b1, b2, c],
    ["learned frontier", "training observations", "new regular observations", "new abnormal observations"],
    loc="upper left",
    prop=matplotlib.font_manager.FontProperties(size=11)
)
plt.xlabel("error train: %d/100 ; errors novel regular: %d/20 ; errors novel abnormal: %d/20" % (n_error_train, n_error_test, n_error_outliers))

plt.show()

As you can see the algorithm was able to detect correctly normal new entities and outliers but made 19 mistakes on the old entities.

# Part III: Proximity-Based Outlier Detection

Proximity-based techniques define a data point as an outlier, if its locality (or proximity) is sparsely populated.
The proximity of a data point may be defined in a variety of ways, which are subtly different from one another, but are similar enough to be considered as one algorithm family.
The most common ways of defining proximity for outlier analysis are as follows:

* **Cluster-based:** The non-membership of a data point in any cluster, its distance from other clusters, and the size of the closest cluster, are used as criteria in order to compute the outlier score. The clustering problem has a complementary relationship to the outlier detection problem, in which points either belong to clusters or outliers.
* **Distance-based:** The distance of a data point to its k-nearest neighbor (or other variant) is used in order to define proximity. Data points with large k-nearest neighbor distances are defined as outliers. Distance-based algorithms typically perform the analysis at a much more detailed granularity than the other two methods. On the other hand, this greater granularity often comes at a significant computational cost.
* **Density-based:** The number of other points within a specified local region (grid region or distance-based region) of a data point,is used in order to define local density. These local density values may be converted into outlier scores. Other kernel-based methods or statistical methods for density estimation may also be used. The major difference between clustering and density-based methods is that clustering methods partition the data points, whereas densitybased methods partition the data space.

## The Local Outlier Factor (LOF)

The LOF algorithm is an unsupervised outlier detection method which computes the local density deviation of a given data point with respect to its neighbors. It considers as outlier samples that have a substantially lower density than their neighbors.

The number of neighbors considered, (parameter n_neighbors) is typically chosen:

    1) greater than the minimum number of objects a cluster has to contain, so that other objects can be local outliers relative to this cluster;

    2) smaller than the maximum number of close by objects that can potentially be local outliers.

In practice, such informations are generally not available, and taking n_neighbors=20 appears to work well in general.

Again initialize parameters for our example:

In [ ]:
n_samples = 200
outliers_fraction = 0.25

Create a coordinate grid:

In [ ]:
xx, yy = np.meshgrid(np.linspace(-7, 7, 100), np.linspace(-7, 7, 100))

Define which part of the sample belongs to inliers/outliers:

In [ ]:
n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)

Define ground-truth labels:

In [ ]:
ground_truth = np.ones(n_samples, dtype=int)
ground_truth[-n_outliers:] = -1

Create an outlier detector

In [ ]:
clf = LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction)

Generate inliers:

In [ ]:
X1 = 0.3 * np.random.randn(n_inliers // 2, 2)
X2 = 0.3 * np.random.randn(n_inliers // 2, 2)

Generate outliers:

In [ ]:
X = np.r_[X1, X2]
X = np.r_[X, np.random.uniform(low=-6, high=6, size=(n_outliers, 2))]

In [ ]:
y_pred = clf.fit_predict(X)
scores_pred = clf.negative_outlier_factor_

In [ ]:
threshold = stats.scoreatpercentile(scores_pred, 100 * outliers_fraction)

In [ ]:
n_errors = (y_pred != ground_truth).sum()

In [ ]:
Z = clf._decision_function(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
Z = Z.reshape(xx.shape)

In [ ]:
plt.title("Outlier/Novelty Detection with Local Outlier Factor (LOF)")
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7))
a = plt.contour(xx, yy, Z, levels=[threshold], linewidths=2, colors='red')
plt.contourf(xx, yy, Z, levels=[threshold, Z.max()], colors='orange')
b = plt.scatter(X[:-n_outliers, 0], X[:-n_outliers, 1], c='white', s=20, edgecolor='k')
c = plt.scatter(X[-n_outliers:, 0], X[-n_outliers:, 1], c='black', s=20, edgecolor='k')
plt.axis('tight')
plt.legend(
    [a.collections[0], b, c],
    ['learned decision function', 'true inliers', 'true outliers'],
    prop=matplotlib.font_manager.FontProperties(size=10),
    loc='lower right'
)
plt.xlabel("%s (errors: %d)" % ("Local Outlier Factor (LOF)", n_errors))
plt.xlim((-7, 7))
plt.ylim((-7, 7))

plt.show()